In [ ]:
import fitz  # PyMuPDF
import re
import pandas as pd
from pathlib import Path
from openai import OpenAI

# --------------------- Configuration ---------------------
OPENROUTER_API_KEY = "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"   # Replace with your key
MODEL_NAME = "google/gemma-3-27b-it:free"
PDF_FOLDER = Path(r"C:\Users\User\Downloads\student_pdf")  # Update this folder path

# --------------------- Sample Answers (Key) ---------------------
sample_answers = {
    1: "Climate change leads to global warming, rising sea levels, and changes in weather patterns.",
    2: "Artificial Intelligence simulates human intelligence. Examples include Siri, Google Maps, and ChatGPT.",
    3: "Photosynthesis is the process in which plants use sunlight, carbon dioxide, and water to produce food and oxygen.",
    4: "The water cycle includes evaporation, condensation, precipitation, and collection.",
    5: "Democracy is a system of government by the people, through elected representatives, ensuring equal rights."
}

# --------------------- Model Grading Call ---------------------
def call_genai_grader(student_answer, expected_answer):
    """
    Calls Gemma model through OpenRouter to assign a score (0-4) based on correctness and completeness.
    """
    prompt = (
        f"Score the student's answer from 0 to 4 based on relevance, completeness, and correctness.\n"
        f"Expected answer: {expected_answer}\n"
        f"Student answer: {student_answer}\n"
        f"Reply ONLY with a number from 0 to 4."
    )

    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=OPENROUTER_API_KEY
    )

    try:
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=5
        )
        score_text = response.choices[0].message.content.strip()
        return int(re.search(r"[0-4]", score_text).group())
    except Exception as e:
        print(f"Error while grading: {e}")
        return 0

# --------------------- PDF Parsing ---------------------
def extract_answers_from_pdf(pdf_path):
    """
    Extracts answers from the student's PDF.
    Supports Q1/A1 format and paragraph-style answers.
    """
    doc = fitz.open(pdf_path)
    text = " ".join([page.get_text() for page in doc])
    doc.close()

    # Try to match answers starting with A1:, A2: etc.
    answers = re.findall(r"A\d+:\s*(.*?)(?=Q\d+:|A\d+:|\Z)", text, re.DOTALL)

    # If no matches, return the entire text as one answer (paragraph style)
    if not answers:
        return [text.strip()]
    return [a.strip() for a in answers]

# --------------------- Main Processing ---------------------
def process_all_pdfs(pdf_folder):
    """
    Process all PDFs in the folder and calculate scores.
    """
    results = []
    pdf_files = sorted(pdf_folder.glob("*.pdf"))

    print(f"Found {len(pdf_files)} PDFs in {pdf_folder}")

    for idx, pdf in enumerate(pdf_files):
        student_id = f"STUDENT_{idx+1:03}"
        student_answers = extract_answers_from_pdf(pdf)

        total_score = 0
        question_wise = []

        for q_no in range(1, len(sample_answers)+1):
            student_ans = student_answers[q_no - 1] if q_no <= len(student_answers) else ""
            model_ans = sample_answers[q_no]
            score = call_genai_grader(student_ans, model_ans)
            total_score += score

            question_wise.append({
                "Q": q_no,
                "Student Answer": student_ans,
                "Expected Answer": model_ans,
                "Score": score
            })

        remark = "Good" if total_score >= 9 else "Needs Improvement"

        results.append({
            "Student Name": student_id,
            "Score": total_score,
            "Remark": remark,
            "Details": question_wise
        })

    return results

# --------------------- Output Results ---------------------
def generate_dataframe(results):
    """
    Create a DataFrame with total scores and rank.
    """
    if not results:
        print("No results to display. Check if PDFs were processed correctly.")
        return pd.DataFrame()

    df = pd.DataFrame([{k: v for k, v in r.items() if k != "Details"} for r in results])
    df["Rank"] = df["Score"].rank(method='min', ascending=False).astype(int)
    df = df.sort_values(by="Rank").reset_index(drop=True)
    return df

# --------------------- Run ---------------------
if __name__ == "__main__":
    final_results = process_all_pdfs(PDF_FOLDER)
    df_result = generate_dataframe(final_results)

    print("\nFinal Rank List:\n")
    print(df_result)

    for r in final_results:
        print(f"\n===== {r['Student Name']} =====")
        for q in r["Details"]:
            print(f"Q{q['Q']}\nStudent Answer: {q['Student Answer']}\nExpected Answer: {q['Expected Answer']}\nScore: {q['Score']}\n")


In [ ]:
''' What is LMS Integration?
In a real-world LMS (e.g., Moodle, Blackboard, Google Classroom), after evaluating student submissions, you push the results (scores, feedback) to the LMS system so that:

Students can see their marks in their LMS portal.

Teachers can track reports and analytics.

This is usually done using:

API Calls → sending data via HTTP requests (REST/GraphQL)

Database Integration → directly inserting results into the LMS database (less common for security reasons).

3. How Can We Integrate with LMS in This Code?
You just need to add a function like this:

python
Copy
Edit
import requests

def send_to_lms(student_id, score, remark, details):
    """
    Sends student results to LMS via an API.
    """
    api_url = "https://lms.example.com/api/results"   # Example LMS API endpoint
    payload = {
        "student_id": student_id,
        "score": score,
        "remark": remark,
        "details": details
    }
    headers = {"Authorization": "Bearer YOUR_LMS_API_KEY"}
    
    response = requests.post(api_url, json=payload, headers=headers)
    
    if response.status_code == 200:
        print(f"LMS updated for {student_id}")
    else:
        print(f"Failed to update LMS for {student_id}: {response.text}")
Then call it at the end of your loop:

python
Copy
Edit
for r in final_results:
    send_to_lms(r["Student Name"], r["Score"], r["Remark"], r["Details"])'''